In [1]:
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.io as pio
from plotly.subplots import make_subplots

import plotly.graph_objects as go

pio.renderers.default = "browser"


In [2]:
# Load the data

user_rsc = pd.read_csv(f'datasets/user_component_randm.csv')
total_inst = pd.read_csv(f'datasets/total_instances.csv')
strg = pd.read_csv(f'datasets/strg_price.csv')
cpu_ram = pd.read_csv(f'datasets/cpu_ram_price.csv')
strg_pred = pd.read_csv('datasets/strg_predictions.csv')


In [3]:
# Rename dataframes' columns
strg = strg.rename(columns={'Unnamed: 0': 'Cluster'})
cpu_ram = cpu_ram.rename(columns={'Unnamed: 0': 'Label'})
user_rsc = user_rsc.rename(columns={'size': 'Cluster'})
user_rsc = user_rsc.drop(columns='Unnamed: 0')
total_inst = total_inst.drop(columns='Unnamed: 0')
strg_pred = strg_pred.drop(columns='Unnamed: 0')


In [4]:
# Check between two dataframes, if the unique number of users have the same shape and elements
np.array_equal(strg_pred.users.unique(), strg_pred.users.unique())

True

In [5]:

total_inst['iaas_cpu_ram'] = total_inst['iaas instances'] * cpu_ram['Mean_price'][1]
total_inst['caas_cpu_ram'] = total_inst['caas instances'] * cpu_ram['Mean_price'][0]

total_inst = total_inst.rename(columns={'iaas_cpu_ram':'IaaS', 'caas_cpu_ram':'CaaS'})

In [6]:
# Plot Users - CPU RAM price 

total_inst = total_inst.sort_values(by='users')
fig = px.scatter(total_inst, x='users', y=['CaaS', 'IaaS'], opacity=0.7,
             trendline='ols', trendline_scope='trace')
results_cpu = px.get_trendline_results(fig)
print(results_cpu)

fig.update_layout(
    title="CPU & RAM price IaaS vs Caas",
    xaxis_title="Users",
    yaxis_title="Price per Hour($/hour)",
    legend_title="Cloud service design",
    legend=dict(
    yanchor="top",
    y=0.99,
    xanchor="left",
    x=0.01
),
    font=dict(
        family="Courier New, monospace",
        size=32,
        color="RebeccaPurple"
    )
)
              
fig.show()

  variable                                     px_fit_results
0     CaaS  <statsmodels.regression.linear_model.Regressio...
1     IaaS  <statsmodels.regression.linear_model.Regressio...


In [7]:
cpu_iaas_slope=results_cpu.px_fit_results.iloc[1].summary()
cpu_iaas_slope

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       1.000
Model:                            OLS   Adj. R-squared:                  1.000
Method:                 Least Squares   F-statistic:                 1.209e+06
Date:                Sun, 17 Jul 2022   Prob (F-statistic):               0.00
Time:                        01:35:55   Log-Likelihood:                -219.49
No. Observations:                 400   AIC:                             443.0
Df Residuals:                     398   BIC:                             451.0
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          1.9041      0.025     74.789      0.000       1.854       1.954
x1             0.0011   9.93e-07   1099.676      0.000       0.001       0.001
==============================================================================
Omnibus:                        6.695   Durbin-Watson:                   0.438
Prob(Omnibus):                  0.035   Jarque-Bera (JB):                4.787
Skew:                           0.134   Prob(JB):                       0.0913
Kurtosis:                       2.536   Cond. No.                     3.11e+04
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 3.11e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [8]:
cpu_caas_slope=results_cpu.px_fit_results.iloc[0].summary()
cpu_caas_slope

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       1.000
Model:                            OLS   Adj. R-squared:                  1.000
Method:                 Least Squares   F-statistic:                 1.209e+06
Date:                Sun, 17 Jul 2022   Prob (F-statistic):               0.00
Time:                        01:35:55   Log-Likelihood:                 301.45
No. Observations:                 400   AIC:                            -598.9
Df Residuals:                     398   BIC:                            -590.9
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.5177      0.007     74.789      0.000       0.504       0.531
x1             0.0003    2.7e-07   1099.676      0.000       0.000       0.000
==============================================================================
Omnibus:                        6.695   Durbin-Watson:                   0.438
Prob(Omnibus):                  0.035   Jarque-Bera (JB):                4.787
Skew:                           0.134   Prob(JB):                       0.0913
Kurtosis:                       2.536   Cond. No.                     3.11e+04
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 3.11e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [9]:
strg

,Cluster,Storage_min,Storage_max,Disk_type,Mean_price,Median_price,Standard_deviation
0,high-speed,100,500,['HDD'],0.017612,0.01590,0.013685
1,low-capacity,100,500,['SSD'],0.046262,0.01388,0.065205
2,high-capacity,800,1000,['HDD' 'SSD'],0.147184,0.03125,0.183875
3,low-speed,2000,2000,['HDD' 'SSD'],0.327060,0.06380,0.308295


In [10]:
# Multiply the mean price of each Storage Cluster with the total storage per each component
# https://stackoverflow.com/questions/70445040/compare-the-values-of-two-columns-of-different-length-in-two-different-dataframe
strg_pred['STORAGE'] *= strg_pred.merge(strg, on=['Cluster'], how='left')['Mean_price']
strg_pred.rename(columns={'STORAGE': 'Storage Price'})

,users,component,instances,RAM,CPU,Storage Price,Disk_type,technology,Cluster
0,576,VMfe,1,17,13,12.375094,SSD,iaas,low-capacity
1,576,VMbe,1,33,9,10.611048,HDD,iaas,high-speed
2,576,VMdb,1,25,9,4.741859,SSD,iaas,low-capacity
3,960,VMfe,1,17,13,12.375094,SSD,iaas,low-capacity
4,960,VMbe,1,33,9,10.611048,HDD,iaas,high-speed
...,...,...,...,...,...,...,...,...,...
2795,26324,S4,3,24,12,0.528351,HDD,caas,high-speed
2796,55427,S1,55,440,440,4587.016500,SSD,caas,low-speed
2797,55427,S2,1,32,8,10.567018,HDD,caas,high-speed
2798,55427,S3,12,288,96,176.621229,SSD,caas,high-capacity


In [11]:
strg_pred.query("technology == 'iaas'").replace({'nginx': 'VMfe', 'framework': 'VMbe', 'rel_db': 'VMdb'})

,users,component,instances,RAM,CPU,STORAGE,Disk_type,technology,Cluster
0,576,VMfe,1,17,13,12.375094,SSD,iaas,low-capacity
1,576,VMbe,1,33,9,10.611048,HDD,iaas,high-speed
2,576,VMdb,1,25,9,4.741859,SSD,iaas,low-capacity
3,960,VMfe,1,17,13,12.375094,SSD,iaas,low-capacity
4,960,VMbe,1,33,9,10.611048,HDD,iaas,high-speed
...,...,...,...,...,...,...,...,...,...
1195,26324,VMbe,1,33,9,10.611048,HDD,iaas,high-speed
1196,26324,VMdb,6,150,54,28.451151,SSD,iaas,low-capacity
1197,55427,VMfe,55,935,715,4811.870250,SSD,iaas,low-speed
1198,55427,VMbe,1,33,9,10.611048,HDD,iaas,high-speed


In [12]:
# Group by users and technology and sum the Storage
strg_price = pd.DataFrame(strg_pred.groupby(['users','technology'])['STORAGE'].sum())


In [13]:
strg_price = strg_price.rename(columns={'STORAGE': 'Storage Price'})
strg_price.reset_index(inplace=True, level=['users', 'technology'])

strg_price

,users,technology,Storage Price
0,10,caas,27.166158
1,10,iaas,27.728000
2,12,caas,27.166158
3,12,iaas,27.728000
4,17,caas,27.166158
...,...,...,...
795,77847,iaas,7450.353901
796,79149,caas,7206.549565
797,79149,iaas,7537.842451
798,79712,caas,7206.549565


In [14]:
# Plot Users - Storage price 
fig = px.scatter(strg_price, x='users', y='Storage Price', color='technology', log_y=False, opacity=0.7,
             trendline='ols', trendline_scope='trace')
results_stg = px.get_trendline_results(fig)

fig.update_layout(
    title="Storage Price IaaS vs CaaS",
    xaxis_title="Users",
    yaxis_title="Storage Price ($/GB)",
    legend_title="Cloud service design",
    legend=dict(
    yanchor="top",
    y=0.99,
    xanchor="left",
    x=0.01
),
    font=dict(
        family="Courier New, monospace",
        size=32,
        color="RebeccaPurple"
    )
)


fig.show()

In [15]:
stg_iaas_slope=results_stg.query("technology == 'iaas'").px_fit_results.iloc[0].summary()
stg_iaas_slope

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.998
Model:                            OLS   Adj. R-squared:                  0.998
Method:                 Least Squares   F-statistic:                 1.762e+05
Date:                Sun, 17 Jul 2022   Prob (F-statistic):               0.00
Time:                        01:35:55   Log-Likelihood:                -2374.0
No. Observations:                 400   AIC:                             4752.
Df Residuals:                     398   BIC:                             4760.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const        -63.1599      5.510    -11.463      0.000     -73.992     -52.328
x1             0.0932      0.000    419.787      0.000       0.093       0.094
==============================================================================
Omnibus:                        0.223   Durbin-Watson:                   0.120
Prob(Omnibus):                  0.895   Jarque-Bera (JB):                0.095
Skew:                          -0.016   Prob(JB):                        0.954
Kurtosis:                       3.068   Cond. No.                     2.98e+04
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.98e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [16]:
stg_caas_slope=results_stg.query("technology == 'caas'").px_fit_results.iloc[0].summary()
stg_caas_slope

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.998
Model:                            OLS   Adj. R-squared:                  0.998
Method:                 Least Squares   F-statistic:                 1.717e+05
Date:                Sun, 17 Jul 2022   Prob (F-statistic):               0.00
Time:                        01:35:55   Log-Likelihood:                -2360.9
No. Observations:                 400   AIC:                             4726.
Df Residuals:                     398   BIC:                             4734.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const        -60.3765      5.332    -11.324      0.000     -70.858     -49.895
x1             0.0890      0.000    414.346      0.000       0.089       0.089
==============================================================================
Omnibus:                        0.679   Durbin-Watson:                   0.119
Prob(Omnibus):                  0.712   Jarque-Bera (JB):                0.466
Skew:                           0.021   Prob(JB):                        0.792
Kurtosis:                       3.162   Cond. No.                     2.98e+04
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.98e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [17]:
price_all = strg_price.sort_values(by=['technology', 'users'])
# price_all = strg_price.sort_values(by='users', inplace=True)
# price_all['CPU'] = (price_all['technology'] == 'iaas'|total_inst['caas_cpu_ram'])


i = price_all[price_all['technology'] == 'iaas']
i = i.reset_index()
i['CPU_RAM Price'] = total_inst['iaas_cpu_ram']

c = price_all[price_all['technology'] == 'caas']
c = c.reset_index()
c['CPU_RAM Price'] = total_inst['caas_cpu_ram']

frames = [i, c]
price_all = pd.concat(frames, ignore_index=True)


KeyError: 'iaas_cpu_ram'

In [ ]:
price_all = price_all.drop('index', axis=1)
price_all

In [ ]:
# # Plot Users - Storage price 


# price_all = strg_price.sort_values(by=['users'])

# fig = make_subplots(rows=2, cols=1)
# fig.append_trace(go.Scatter.line(x=price_all['users'], y=price_all['Storage Price']), row=1, col=1)
# fig.append_trace(go.Line(x=price_all['users'], y=price_all['CPU_RAM Price']), row=2, col=1)

# # = px.line(price_all, x='users', y='Storage Price', color='technology', title='Storage Price IaaS vs CaaS')
              
# fig.show()